In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from plotly import graph_objects as go
import plotly as py

from datetime import datetime
from datetime import timedelta

import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

from sklearn.utils import resample

# 1 - Classificar o dataset clusterizado por dia

In [2]:
df_cluster = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/clustered_data.csv', sep = ';')
df_cluster

,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Mes,Cluster,OrdensServico,LocalMax,Label
0,65.679216,735.558902,16.414147,19.501318,13.894332,116.433333,114.439583,2.144792,1.52,0,1,0,0.0,0,0
1,76.017708,734.376875,15.638204,18.355709,14.988855,40.879167,162.916667,1.103542,139.12,0,1,0,0.0,0,0
2,73.900711,733.959619,16.060072,18.713147,15.064426,59.225559,102.732787,1.328760,68.92,0,1,0,0.0,0,0
3,73.177223,734.321458,17.005510,19.832342,15.797179,82.889583,85.015766,1.597480,62.36,0,1,0,0.0,0,0
4,64.092344,736.860417,18.614403,21.592730,15.512999,161.429167,150.189583,2.222917,11.96,0,1,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065,58.462548,924.237189,20.782350,24.296560,11.613997,86.694988,154.352083,8.483779,0.00,1,6,0,1.0,1,0
3066,78.492677,928.165211,19.225816,23.262681,15.223413,58.906011,125.716667,4.999583,0.32,0,6,0,0.0,0,0
3067,66.947345,928.851826,21.216116,24.902792,14.681483,82.298725,89.068750,7.688742,0.00,0,6,0,0.0,0,0
3068,54.981241,927.484876,21.871250,25.403125,11.656711,92.199144,143.372917,6.376733,0.00,0,6,0,0.0,0,0


In [3]:
df_cluster['Ordens'] = df_cluster['OrdensServico']
df_cluster.loc[df_cluster['Ordens'] >= 1, 'Ordens'] = 1
df_cluster = df_cluster.drop(columns=['Label', 'OrdensServico', 'LocalMax'])

In [4]:
xgb = xgboost.XGBClassifier()

cols_rem = ['Ordens', 'Cluster']

x = df_cluster[[c for c in df_cluster.columns if c not in cols_rem]]
y = df_cluster['Ordens']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378)

# concatenate our training data back together
X = pd.concat([x_treino, y_treino], axis=1)

# separate minority and majority classes
not_ordem = X[X['Ordens']==0].copy()
ordem = X[X['Ordens']==1].copy()

# upsample minority
ordem_upsampled = resample(ordem,
                        replace=True, # sample with replacement
                        n_samples=len(not_ordem), # match number in majority class
                        random_state=378) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_ordem, ordem_upsampled])

x_treino = upsampled[[c for c in df_cluster.columns if c not in cols_rem]]
y_treino = upsampled['Ordens']

display(y_treino.value_counts())

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':10, 'eta':2, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
display(confusion_matrix(y_teste, y_teste_pred,))

1.0    1974
0.0    1974
Name: Ordens, dtype: int64

Treino: 0.9485815602836879
Teste: 0.9022801302931596
Precisão: 0.3373493975903614
Recall: 0.4444444444444444
F1: 0.3835616438356164


array([[0.93589744, 0.06410256],
       [0.55555556, 0.44444444]])

array([[803,  55],
       [ 35,  28]], dtype=int64)

In [5]:
df_cluster.groupby(['Ordens']).count()

,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Mes,Cluster
Ordens,,,,,,,,,,,
0.0,2832,2832,2832,2832,2832,2832,2832,2832,2832,2832,2832
1.0,238,238,238,238,238,238,238,238,238,238,238


# 2 - Classificar o dataset clusterizado por 15 mins

In [6]:
df_clustered_total = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/clustered.csv', sep = ';')
df_clustered_total

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,OrdensServico,Cluster,LocalMax,Label
0,2011-01-01 00:00:00,Camilopolis,84.8,920.6,20.200000,22.8,17.500000,0.000000,137.0,1.5,0.0,0,0.0,0,0,0
1,2011-01-01 00:00:00,RM,86.6,926.7,20.100000,23.3,17.800000,0.000000,131.0,2.5,0.0,0,0.0,0,0,0
2,2011-01-01 00:00:00,Paraiso,92.3,919.1,19.300000,22.7,18.000000,0.000000,101.0,2.6,0.0,0,0.0,0,0,0
3,2011-01-01 00:00:00,Erasmo,88.8,920.5,10.243265,22.9,17.700000,0.000000,133.0,0.7,0.0,0,0.0,0,0,0
4,2011-01-01 00:00:00,Vitoria,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,2019-06-30 23:45:00,RM,66.9,931.9,18.800000,22.5,13.300000,0.294461,331.0,0.7,0.0,0,0.0,0,0,0
1463566,2019-06-30 23:45:00,Erasmo,60.7,925.7,19.500000,22.5,11.700000,0.000000,327.0,3.5,0.0,0,0.0,0,0,0
1463567,2019-06-30 23:45:00,Camilopolis,54.4,924.0,20.800000,23.0,12.200000,0.073893,332.0,4.3,0.0,0,0.0,0,0,0
1463568,2019-06-30 23:45:00,Paraiso,55.3,924.2,20.800000,22.1,11.790525,0.000000,25.0,3.6,0.0,0,0.0,0,0,0


In [7]:
df_clustered_total['Ordens'] = df_clustered_total['LocalMax']
df_clustered_total.loc[df_clustered_total['Ordens'] >= 1, 'Ordens'] = 1
df_clustered_total = df_clustered_total.drop(columns=['Label', 'LocalMax', 'OrdensServico'])
df_clustered_total['Data'] = pd.to_datetime(df_clustered_total['Data_Hora'], yearfirst=True).dt.strftime('%Y-%m-%d')
df_clustered_total = df_clustered_total.sort_values(by=['Data_Hora', 'Local'])

In [8]:
df_clustered_total.groupby('Ordens').count()

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Cluster,Data
Ordens,,,,,,,,,,,,,
0,1413590,1413590,1413590,1413590,1413590,1413590,1413590,1413590,1413590,1413590,1413590,1413590,1413590
1,49980,49980,49980,49980,49980,49980,49980,49980,49980,49980,49980,49980,49980


In [9]:
# for i in range(24*4):
#     df_clustered_total['Prec_Shift_' + str(i)] = df_clustered_total['Precipitacao'].shift(-5 * (i+1), fill_value=0)

# df_clustered_total['PrecSum'] = df_clustered_total[[c for c in df_clustered_total.columns if 'Prec_Shift' in c]].sum(axis=1)
# df_clustered_total = df_clustered_total.drop(columns = [c for c in df_clustered_total.columns if 'Prec_Shift' in c])
# df_clustered_total.head(10)

In [10]:
df_prec_sum = df_clustered_total.groupby(['Data', 'Local']).sum().reset_index()[['Data', 'Local', 'Precipitacao']]
df_prec_sum.columns = ['Data', 'Local', 'PrecSum']
df_clustered_total = df_clustered_total.merge(df_prec_sum, on=['Data', 'Local'])
df_clustered_total.head(10)

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Cluster,Data,PrecSum
0,2011-01-01 00:00:00,Camilopolis,84.8,920.6,20.2,22.800000,17.5,0.0,137.0,1.5,0.0,0,0,2011-01-01,0.6
1,2011-01-01 00:15:00,Camilopolis,84.1,920.3,20.1,22.800000,17.4,0.0,76.0,1.9,0.0,0,0,2011-01-01,0.6
2,2011-01-01 00:30:00,Camilopolis,83.9,920.2,20.1,22.900000,17.2,0.0,92.0,2.9,0.0,0,0,2011-01-01,0.6
3,2011-01-01 00:45:00,Camilopolis,83.5,919.9,20.2,22.900000,17.3,0.0,133.0,1.4,0.0,0,0,2011-01-01,0.6
4,2011-01-01 01:00:00,Camilopolis,84.0,919.9,20.1,22.800000,17.3,0.0,151.0,1.8,0.0,0,0,2011-01-01,0.6
5,2011-01-01 01:15:00,Camilopolis,84.5,919.5,20.0,23.143578,17.3,0.0,124.0,1.8,0.0,0,0,2011-01-01,0.6
6,2011-01-01 01:30:00,Camilopolis,84.6,919.4,20.0,23.083792,17.3,0.0,137.0,1.8,0.0,0,0,2011-01-01,0.6
7,2011-01-01 01:45:00,Camilopolis,84.5,919.4,20.0,23.005875,17.3,0.0,115.0,1.9,0.0,0,0,2011-01-01,0.6
8,2011-01-01 02:00:00,Camilopolis,85.8,919.2,19.8,22.935631,17.4,0.0,155.0,1.8,0.0,0,0,2011-01-01,0.6
9,2011-01-01 02:15:00,Camilopolis,86.5,919.0,19.7,22.786449,17.4,0.0,140.0,1.7,0.0,0,0,2011-01-01,0.6


In [11]:
df_clustered_total['Hora'] = pd.to_datetime(df_clustered_total['Data_Hora'], yearfirst=True).dt.hour

df_ohe = df_clustered_total.groupby(['Data', 'Local', 'Hora']).sum().reset_index()[['Data', 'Local', 'Hora', 'Precipitacao']]
s_ohe = df_ohe['Hora']
df_ohe = pd.get_dummies(df_ohe, columns = ['Hora'])
df_ohe['Hora'] = s_ohe

for i in range(24):
    df_ohe.loc[df_ohe['Hora_' + str(i)] == 1, 'Hora_' + str(i)] = df_ohe.loc[df_ohe['Hora_' + str(i)] == 1, 'Precipitacao']

df_clustered_total = df_clustered_total.merge(df_ohe[['Data', 'Local'] + [c for c in df_ohe.columns if 'Hora' in c]], on=['Data', 'Local', 'Hora'])

In [12]:
df_clustered_total['Local'] = df_clustered_total['Local'].rank(method='dense', ascending=False).astype(int)
df_clustered_total_a = df_clustered_total.copy()
df_clustered_total = df_clustered_total.drop(columns = ['Data_Hora', 'Data', 'Hora'])

In [13]:
# df_clustered_total['Local'] = df_clustered_total['Local'].rank(method='dense', ascending=False).astype(int)
# df_clustered_total['Minuto'] = pd.to_datetime(df_clustered_total['Data_Hora'], yearfirst=True).dt.minute
# df_clustered_total = df_clustered_total[df_clustered_total['Minuto'] == 0]
# df_clustered_total = df_clustered_total.drop(columns = ['Data_Hora', 'Minuto'])

In [14]:
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['Cluster'].isin([1,2]))]
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['Precipitacao'] > 10)]
df_slice = df_clustered_total.copy()
#df_slice = df_clustered_total[(df_clustered_total['Cluster'].isin([0]))]
#df_slice.loc[df_slice['Cluster'] == 0, 'Ordens'] = 0
df_slice.loc[(df_slice['Ordens'] == 1) & (df_slice['PrecSum'] <= 10), 'Ordens'] = 0
#df_slice.loc[(df_slice['Ordens'] == 1) & ~((df_clustered_total[[c for c in df_clustered_total.columns if 'Hora_' in c]] <= 20).sum(axis = 1) < 24), 'Ordens'] = 0

In [15]:
df_slice.groupby('Ordens').count()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,...,Hora_14,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23
Ordens,,,,,,,,,,,,,,,,,,,,,
0,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,...,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681
1,32889,32889,32889,32889,32889,32889,32889,32889,32889,32889,...,32889,32889,32889,32889,32889,32889,32889,32889,32889,32889


In [16]:
# Testar prever cluster

xgb = xgboost.XGBClassifier()

cols_rem = ['Ordens', 'Cluster'] + [c for c in df_slice.columns if 'Hora_' in c]

x = df_slice[[c for c in df_slice.columns if c not in cols_rem]]
#x = x.drop(columns = 'Cluster')
y = df_slice['Ordens']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)

# concatenate our training data back together
X = pd.concat([x_treino, y_treino], axis=1)

# separate minority and majority classes
not_ordem = X[X['Ordens']==0].copy()
ordem = X[X['Ordens']==1].copy()

# upsample minority
ordem_upsampled = resample(ordem,
                        replace=True, # sample with replacement
                        n_samples=len(not_ordem), # match number in majority class
                        random_state=378) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_ordem, ordem_upsampled])

x_treino = upsampled[[c for c in df_slice.columns if c not in cols_rem]]
y_treino = upsampled['Ordens']

display(y_treino.value_counts())

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':50, 'eta':1, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
display(confusion_matrix(y_teste, y_teste_pred,))

1    1001477
0    1001477
Name: Ordens, dtype: int64

Treino: 0.9990698737964027
Teste: 0.9956658490312501
Precisão: 0.873545966228893
Recall: 0.9437519002736394
F1: 0.907292833828616


array([[0.9968593, 0.0031407],
       [0.0562481, 0.9437519]])

array([[427856,   1348],
       [   555,   9312]], dtype=int64)

In [17]:
confusion_matrix(y_treino, y_treino_pred, )

# Treino: 0.9995160230081979
# Teste: 0.9987405226034058
# F1: 0.48172446110590444
# array([[0.99882856, 0.00117144],
#        [0.13175676, 0.86824324]])
# array([[438261,    514],
#        [    39,    257]], dtype=int64)

array([[ 999614,    1863],
       [      0, 1001477]], dtype=int64)

In [18]:
cols_rem = ['Cluster'] + [c for c in df_slice.columns if 'Hora_' in c]
df_other = df_clustered_total.loc[(df_clustered_total['Ordens'] == 1) & (df_clustered_total['PrecSum'] <= 10), [c for c in df_slice.columns if c not in cols_rem]]
x_other = df_other.drop(columns = 'Ordens')
y_other = df_other['Ordens']
y_other_pred = bst.predict(xgboost.DMatrix(data=x_other, label=y_other))
y_other_pred
# y_other_pred = [1 if i>0.5 else 0 for i in y_other_pred]
# print(f"Acurácia: {accuracy_score(y_other, y_other_pred)}")
# print(f"Precisão: {precision_score(y_other, y_other_pred)}")
# print(f"Recall: {recall_score(y_other, y_other_pred)}")
# print(f"F1: {f1_score(y_other, y_other_pred)}")
# display(confusion_matrix(y_other, y_other_pred, normalize='true'))
# display(confusion_matrix(y_other, y_other_pred,))

array([0.0416739, 0.0416739, 0.0416739, ..., 0.0416739, 0.0416739,
       0.0416739], dtype=float32)

In [19]:
df_clustered_total_a['Ordens_New'] = df_clustered_total_a['Ordens']
df_clustered_total_a.loc[(df_clustered_total_a['Ordens_New'] == 1) & (df_clustered_total_a['PrecSum'] <= 10), 'Ordens_New'] = 0
df_clustered_total_a['Data_Hora'] = pd.to_datetime(df_clustered_total_a['Data_Hora'], yearfirst=True)

In [20]:
from plotly.subplots import make_subplots
fig = make_subplots(3,1, shared_xaxes=True )

precipitacao_cols = [c for c in df_clustered_total_a.columns if 'Precipitacao' in c]

ano = 2014
df_ano = df_clustered_total_a[df_clustered_total_a['Data_Hora'].dt.year == ano].groupby(['Data', 'Local']).sum().reset_index()
df_ano.loc[df_ano['Ordens'] >= 1, 'Ordens'] = 1
df_ano.loc[df_ano['Ordens_New'] >= 1, 'Ordens_New'] = 1

display(df_ano['Ordens'].value_counts())
display(df_ano['Ordens_New'].value_counts())

for col in precipitacao_cols:
    fig.add_trace(go.Bar(
        x = df_ano['Data'],
        y = df_ano[col],
        name = col,
                            ),
                  row = 1, col = 1
                 )
fig.add_trace(go.Bar(
    x = df_ano['Data'],
    y = df_ano['Ordens'],
    name = 'Ordens de Serviço',
                        ),
                  row = 2, col = 1
             )

fig.add_trace(go.Bar(
    x = df_ano['Data'],
    y = df_ano['Ordens_New'],
    name = 'Label',
                        ),
                  row = 3, col = 1
             )
fig.show()

0    1780
1      40
Name: Ordens, dtype: int64

0    1795
1      25
Name: Ordens_New, dtype: int64

In [21]:
display(df_clustered_total_a['Ordens'].value_counts())
display(df_clustered_total_a['Ordens_New'].value_counts())

0    1413590
1      49980
Name: Ordens, dtype: int64

0    1430681
1      32889
Name: Ordens_New, dtype: int64

In [22]:
df_a = x_teste.copy()
df_a['Label'] = y_teste
df_a['Label_Pred'] = y_teste_pred
df_a

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,PrecSum,Label,Label_Pred
224656,3,85.200000,926.4,18.3,22.600000,15.8,0.0,104.0,2.2,0.0,0.0,0,0
40528,2,40.400000,928.2,30.4,34.000000,15.3,947.0,272.0,0.5,0.0,0.0,0,0
305830,5,41.600000,923.3,28.7,32.200000,14.4,204.0,246.0,5.8,0.0,15.4,0,0
828225,3,100.891975,923.0,18.1,21.854668,18.8,0.0,201.0,4.0,0.8,49.6,0,0
1276303,3,80.100000,931.5,15.2,16.900000,11.8,170.0,142.0,6.8,0.0,4.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70694,2,82.700000,932.9,13.7,18.600000,10.7,64.0,203.0,3.3,0.0,7.8,0,0
319179,5,78.300000,922.3,18.7,23.200000,14.8,73.0,132.0,6.1,0.0,3.2,0,0
842501,3,70.600000,920.0,23.2,28.700000,17.5,372.0,86.0,6.1,0.0,0.0,0,0
1147555,5,38.600000,927.8,21.5,24.300000,6.5,611.0,250.0,14.0,0.0,0.0,0,0


In [23]:
df_a[(df_a['Label'] == 1) & (df_a['Label_Pred'] == 0)]

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,PrecSum,Label,Label_Pred
1222243,3,66.030365,918.9,26.4,30.9,21.605862,149.0,135.0,8.6,0.0,37.2,1,0
859361,2,61.000000,926.7,26.5,34.8,18.300000,46.0,337.0,13.0,0.0,491.9,1,0
850759,2,87.800000,929.6,22.3,27.1,20.400000,18.0,82.0,8.6,0.0,21.6,1,0
1186487,4,81.000000,923.2,22.3,25.8,19.700000,0.0,318.0,8.8,0.0,146.6,1,0
39574,2,46.900000,929.8,28.8,32.8,16.400000,677.0,30.0,2.1,0.0,106.1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
366798,3,101.900000,921.9,19.5,23.9,19.700000,25.0,94.0,6.5,0.0,35.4,1,0
1024343,5,53.900000,921.9,25.6,29.3,15.600000,614.0,280.0,7.2,0.0,42.2,1,0
1449491,2,88.900000,929.4,18.9,22.1,17.800000,94.0,219.0,3.6,1.8,38.8,1,0
692868,4,87.900000,0.0,19.2,21.7,17.100000,0.0,0.0,0.0,3.7,127.3,1,0


In [24]:
df_slice.groupby(['Ordens']).count()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,...,Hora_14,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23
Ordens,,,,,,,,,,,,,,,,,,,,,
0,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,...,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681,1430681
1,32889,32889,32889,32889,32889,32889,32889,32889,32889,32889,...,32889,32889,32889,32889,32889,32889,32889,32889,32889,32889


In [25]:
y_teste.shape[0] / y.shape[0]

0.3

In [26]:
df_slice

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,...,Hora_14,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23
0,5,84.8,920.6,20.2,22.8,17.5,0.0,137.0,1.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,84.1,920.3,20.1,22.8,17.4,0.0,76.0,1.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,83.9,920.2,20.1,22.9,17.2,0.0,92.0,2.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,83.5,919.9,20.2,22.9,17.3,0.0,133.0,1.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,84.0,919.9,20.1,22.8,17.3,0.0,151.0,1.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,1,32.6,925.9,21.7,27.9,5.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1463566,1,34.2,925.9,21.3,27.9,5.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1463567,1,35.2,925.9,21.0,27.7,5.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1463568,1,36.1,925.8,20.8,27.6,5.9,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# import pandas_gbq
# from google.oauth2 import service_account

# PROJECT_ID = 'temporal-285820'
# TABLE_clustered = 'info_pluviometrica.clustered_slice'

# CREDENTIALS = service_account.Credentials.from_service_account_file('../../../key/temporal-285820-cde76c259484.json')
# pandas_gbq.context.credentials = CREDENTIALS

In [28]:
# pandas_gbq.to_gbq(df_slice, TABLE_clustered, project_id=PROJECT_ID, credentials=CREDENTIALS, if_exists='replace')
# print('clustered done!')

In [29]:
df_clustered_total[(df_clustered_total['Cluster'].isin([1,2]))].groupby('Ordens').count()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,...,Hora_14,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23
Ordens,,,,,,,,,,,,,,,,,,,,,
0,28755,28755,28755,28755,28755,28755,28755,28755,28755,28755,...,28755,28755,28755,28755,28755,28755,28755,28755,28755,28755
1,30965,30965,30965,30965,30965,30965,30965,30965,30965,30965,...,30965,30965,30965,30965,30965,30965,30965,30965,30965,30965


# 3 - Tentar prever cluster

In [30]:
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['Cluster'].isin([1,2]))]
df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['PrecSum'] > 10)]
#df_slice = df_clustered_total.copy()
#df_slice.loc[df_slice['Cluster'] == 0, 'Ordens'] = 0
df_slice.loc[(df_slice['Ordens'] == 1) & (df_slice['PrecSum'] <= 10), 'Ordens'] = 0

In [31]:
df_slice['Cluster'].value_counts()

0    293449
1     47937
Name: Cluster, dtype: int64

In [32]:
# Testar prever cluster

xgb = xgboost.XGBClassifier()

cols_rem = ['Ordens', 'Cluster']

x = df_slice[[c for c in df_slice.columns if c not in cols_rem]]
#x = x.drop(columns = 'Cluster')
y = df_slice['Cluster']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)

# concatenate our training data back together
X = pd.concat([x_treino, y_treino], axis=1)

# separate minority and majority classes
zero = X[X['Cluster']==0].copy()
one = X[X['Cluster']==1].copy()
# two = X[X['Cluster']==2].copy()
# three = X[X['Cluster']==3].copy()

# upsample minority
# three_upsampled = resample(three,
#                         replace=True, # sample with replacement
#                         n_samples=len(zero), # match number in majority class
#                         random_state=378) # reproducible results

# # upsample minority
# two_upsampled = resample(two,
#                         replace=True, # sample with replacement
#                         n_samples=len(zero), # match number in majority class
#                         random_state=378) # reproducible results

# upsample minority
one_upsampled = resample(one,
                        replace=True, # sample with replacement
                        n_samples=len(zero), # match number in majority class
                        random_state=378) # reproducible results

# combine majority and upsampled minority
#upsampled = pd.concat([zero, one_upsampled, two_upsampled, three_upsampled])
upsampled = pd.concat([zero, one_upsampled])

x_treino = upsampled[[c for c in df_slice.columns if c not in cols_rem]]
y_treino = upsampled['Cluster']

display(y_treino.value_counts())

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':50, 'eta':1, 'objective':'multi:softmax', 'num_class': 4, 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
#y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
#y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

# print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
# print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
# print(f"Precisão: {precision_score(y_teste, y_teste_pred, average='macro')}")
# print(f"Recall: {recall_score(y_teste, y_teste_pred, average='macro')}")
# print(f"F1: {f1_score(y_teste, y_teste_pred, average='macro')}")
# display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
# display(confusion_matrix(y_teste, y_teste_pred,))

print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
display(confusion_matrix(y_teste, y_teste_pred,))

1    205414
0    205414
Name: Cluster, dtype: int64

Treino: 0.9972494571937648
Teste: 0.9767614435244493
Precisão: 0.9009689274974941
Recall: 0.9375564981572909
F1: 0.9188986573979419


array([[0.98316579, 0.01683421],
       [0.0624435 , 0.9375565 ]])

array([[86553,  1482],
       [  898, 13483]], dtype=int64)

In [33]:
df_m = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/merged.csv', sep = ';')
df_m['Data_Hora'] = pd.to_datetime(df_m['Data_Hora'], yearfirst=True)
df_m['Data'] = df_m['Data_Hora'].dt.strftime('%Y-%m-%d')

In [34]:
df_m.groupby('Data').sum()[[c for c in df_m.columns if 'Precipitacao' in c]].reset_index()

,Data,Precipitacao_0,Precipitacao_1,Precipitacao_2,Precipitacao_3,Precipitacao_4
0,2011-01-01,0.6,2.2,1.6,3.2,0.0
1,2011-01-02,183.0,181.2,161.4,170.0,0.0
2,2011-01-03,87.0,84.8,99.8,73.0,0.0
3,2011-01-04,58.4,124.6,48.8,80.0,0.0
4,2011-01-05,15.6,14.8,13.0,16.4,0.0
...,...,...,...,...,...,...
3191,2019-09-27,0.0,0.0,1.6,1.6,0.8
3192,2019-09-28,0.0,0.0,0.8,0.8,0.8
3193,2019-09-29,0.0,0.0,0.0,0.8,0.8
3194,2019-09-30,0.0,0.0,0.0,0.0,0.0


In [35]:
df_clustered_total.groupby('Cluster').sum()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,...,Hora_14,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23
Cluster,,,,,,,,,,,,,,,,,,,,,
0,4211550,9.395990e+07,1.173991e+09,2.568940e+07,3.048112e+07,1.857269e+07,1.613160e+08,1.830458e+08,9.898782e+06,190690.9,...,33721.6,48141.4,51679.3,47861.3,48000.0,41426.5,39828.2,35614.0,39016.6,40797.3
1,179160,4.009734e+06,4.933262e+07,1.206003e+06,1.413498e+06,9.198721e+05,6.372681e+06,8.322925e+06,3.896719e+05,53694.0,...,6660.8,13655.5,18105.2,24388.0,25119.2,21564.8,18369.1,16514.4,10314.8,8451.6
